In [1]:
import os

In [2]:
%pwd

'd:\\Text-Summarizer-Project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\Text-Summarizer-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)

class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
import sys
sys.path.append("D:\Text-Summarizer-Project\src\TextSummarizer")


In [7]:
!pip install TextSummarizer

In [6]:
from src.TextSummarizer.constants import *

In [13]:
%pip install boxsdk

Note: you may need to restart the kernel to use updated packages.


In [26]:
%pip install --upgrade python-box 
%pip install PyYAML
%pip install ensure

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
from src.TextSummarizer.utils.common import read_yaml, create_dir

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_dir([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_dir([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

        

In [9]:
import os
import urllib.request as request
import zipfile
from TextSummarizer.logging import logger
from TextSummarizer.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url = self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download. with the following information: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip file path: str
        Extracts the zip file into data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-09-26 19:39:57,387: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-26 19:39:57,389: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-26 19:39:57,392: INFO: common: created directory at :artifacts]
[2023-09-26 19:39:57,393: INFO: common: created directory at :artifacts/data_ingestion]
[2023-09-26 19:39:59,485: INFO: 67138302: artifacts/data_ingestion/data.zip download. with the following information: 
Connection: close
Content-Length: 8398633
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "aa5a350a28b84f211c0a59db96f426d3ee0bef96d02029772095031deb8c9faf"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: E584:3C55:1296BF:16CBAB:65136BCE
Accept-Ranges: bytes
Date: Tue, 26 Sep 2023 23:39:58 GMT
Via: 1.1 varnish
X-Served-By: cache-yyz4540-YYZ
X-Ca